In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import os

In [2]:
folder = 'src/data'
os.listdir(folder)

['aisles.csv',
 'departments.csv',
 'orders.csv',
 'order_products__prior.csv',
 'order_products__train.csv',
 'products.csv']

In [3]:
products = pd.read_csv(folder + '/' + 'products.csv'); products

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13
...,...,...,...,...
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5
49684,49685,En Croute Roast Hazelnut Cranberry,42,1
49685,49686,Artisan Baguette,112,3
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8


In [4]:
departments = pd.read_csv(folder + '/' + 'departments.csv')
departments

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol
5,6,international
6,7,beverages
7,8,pets
8,9,dry goods pasta
9,10,bulk


In [5]:
df = pd.merge(products, departments, how="left", on="department_id")[["product_id","product_name", "department"]]
df.head()

,product_id,product_name,department
0,1,Chocolate Sandwich Cookies,snacks
1,2,All-Seasons Salt,pantry
2,3,Robust Golden Unsweetened Oolong Tea,beverages
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,frozen
4,5,Green Chile Anytime Sauce,pantry
...,...,...,...
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla",alcohol
49684,49685,En Croute Roast Hazelnut Cranberry,frozen
49685,49686,Artisan Baguette,bakery
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,pets


In [6]:
db_uri = "blablabla"

In [7]:
from sqlmodel import SQLModel, Field, create_engine, Session
class Product(SQLModel, table=True):
    product_id: int = Field(default=None, primary_key=True) 
    product_name: str 
    department: str
    

engine = create_engine(db_uri)
SQLModel.metadata.create_all(bind=engine)

In [8]:
def df_to_sqlmodel(df: pd.DataFrame):
    objs = [Product(**row) for row in df.to_dict('records')]
    return objs

In [9]:
df_to_sqlmodel(df)

[Product(product_id=1, product_name='Chocolate Sandwich Cookies', department='snacks'),
 Product(product_id=2, product_name='All-Seasons Salt', department='pantry'),
 Product(product_id=3, product_name='Robust Golden Unsweetened Oolong Tea', department='beverages'),
 Product(product_id=4, product_name='Smart Ones Classic Favorites Mini Rigatoni With Vodka Cream Sauce', department='frozen'),
 Product(product_id=5, product_name='Green Chile Anytime Sauce', department='pantry'),
 Product(product_id=6, product_name='Dry Nose Oil', department='personal care'),
 Product(product_id=7, product_name='Pure Coconut Water With Orange', department='beverages'),
 Product(product_id=8, product_name="Cut Russet Potatoes Steam N' Mash", department='frozen'),
 Product(product_id=9, product_name='Light Strawberry Blueberry Yogurt', department='dairy eggs'),
 Product(product_id=10, product_name='Sparkling Orange Juice & Prickly Pear Beverage', department='beverages'),
 Product(product_id=11, product_name=

In [10]:
with Session(engine) as sess:
    for row in df_to_sqlmodel(df):
        sess.add(row)
    
    sess.commit()
